In [419]:
#Import Libraries
import pygame
import random
import math
import time
import datetime
import numpy as np
import schedule

import cv2
import matplotlib

In [420]:
#Define Player class

class Player:
    def __init__(self, x, y, radius, color):
        self.x = x #set x-coordinate
        self.y = y #set y-coordinate
        self.radius = radius #set radius
        self.color = color #set player color
        self.speed = random.uniform(0.2, 1.7) #set random speed
        self.angle = random.uniform(0, 2 * math.pi) #set random angle
        self.last_update_time = time.time() #set last update time
        self.angle_update = False #Set angle update for basketball displacement
    
    #Update speed and angle of the player
    def update_speed_angle(self, basketball):
            if time.time() - self.last_update_time >= random.uniform(3, 6):
                
                #After angle has updated, change the angle_update and change_displacement to false
                self.angle_update = False
                basketball.change_displacement = False
                
                #Change speed
                self.speed = random.uniform(0.2, 1.7)
                
                #Change angle
                if int(time.time()%60)%2 == 0: #For even distribution, of adding/subtracting self.angle
                    self.angle += np.radians(np.random.normal(90, 1.5))
                else:
                    self.angle -= np.radians(np.random.normal(90, 1.5))
                self.last_update_time = time.time()
    
    #Move player
    def move(self, basketball):
        self.update_speed_angle(basketball)
        
        self.x += self.speed * math.cos(self.angle)
        self.y += self.speed * math.sin(self.angle)
        
        # Wall collision
        if self.x + self.radius > SCREEN_WIDTH-15 or self.x - self.radius < 0:
            self.angle = math.pi - self.angle
        if self.y + self.radius > SCREEN_HEIGHT-15 or self.y - self.radius < 0:
            self.angle = -self.angle
    
    #Draw player
    def draw(self):
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), self.radius)
        

In [421]:
#Define Basketball class

class Basketball:
    def __init__(self, x, y, radius, color):
        self.x = x #set x-coordinate
        self.y = y #set y-coordinate
        self.radius = radius #set basketball radius
        self.color = color #set basketball color
        self.speed = random.uniform(0.2, 1.7) #set random speed
        self.angle = random.uniform(0, 2 * math.pi) #set random angle
        self.last_update_time = time.time()  #set last update time
        self.history = [] # To store positions for fluctuation effect
        self.offset = 0 #Offset refers to the displacment of basketball from the player. Initialize at 0, since this will update later.
        self.change_displacement = False #To prevent constant change in displacment from the player
    
    #Update position of basketball
    def update_position(self, dribbling_player, elapsed_time):
        # Oscillation parameters
        amplitude = 20  # Amplitude of the oscillation
        frequency = 8  # Frequency of the oscillation

        # Oscillation using a sine function to create a dynamic, naturalistic movement
        self.offset = amplitude * math.sin(frequency * elapsed_time)

        # Calculate basketball's position
        self.x, self.y, self.change_displacement, dribbling_player.angle_update = update_basketball_position(self, dribbling_player, 20)

    #Draw basketball 
    def draw(self):
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), self.radius)

In [422]:
#Displace basketball position relative to current player to make a more natural dribbling motion

def update_basketball_position(ball, player, displacement):
    
    #Ball should change location if and change displacement player hasnt changed direction 
    if player.angle_update == False and ball.change_displacement == False:
        # Choose randomly between adding or subtracting (0 for add, 1 for subtract)
        displacement_randomness = random.choice([0, 1])

        # Determine sides based on current player's coordinates (x, y)
        right_side = math.cos(player.x) > math.sin(player.y)
        left_side = math.cos(player.x) < -(math.sin(player.y))
        
        # Calculate new position based on the conditions provided
        if right_side and left_side:
            if displacement_randomness == 0:
                ball.x = player.x + displacement + ball.offset * math.cos(ball.angle)
            else:
                ball.x = player.x - displacement + ball.offset * math.cos(ball.angle)
        elif right_side and not left_side:
            if displacement_randomness == 0:
                ball.y = player.y + displacement + ball.offset * math.sin(ball.angle)
            else:
                ball.y = player.y - displacement + ball.offset * math.sin(ball.angle)
        elif not right_side and left_side:
            if displacement_randomness == 0:
                ball.y = player.y + displacement + ball.offset * math.sin(ball.angle)
            else:
                ball.y = player.y - displacement + ball.offset * math.sin(ball.angle)
        else:  # not right_side and not left_side
            if displacement_randomness == 0:
                ball.x = player.x + displacement + ball.offset * math.cos(ball.angle)
            else:
                ball.x = player.x - displacement + ball.offset * math.cos(ball.angle)

        ball.change_displacement = True
        player.angle_update = True
    
    else:
        ball.x = player.x + displacement + ball.offset * math.cos(ball.angle)
        ball.y = player.y + displacement + ball.offset * math.sin(ball.angle)

        
    return ball.x, ball.y, ball.change_displacement, player.angle_update

In [423]:
# Simulate the basketball being passed
def move_basketball_to_player(basketball, target, players, exclusion_list, speed):
    
    adjusted_movement = adjust_path_if_needed(basketball, target, players, exclusion_list)
    basketball.x += adjusted_movement.x
    basketball.y += adjusted_movement.y

    # Check if reached target -> Distance between player and basketball
    if pygame.math.Vector2(target.x - basketball.x, target.y - basketball.y).length() <= speed:
        if ((abs(basketball.y - target.y) <= 5) and (abs(basketball.x - target.x) <= 5)):
            return True
    else:
        return False

In [424]:
# Adjust basketball pass path

def adjust_path_if_needed(basketball, target, players, exclusion_list):

    # Calculate the direct path vector
    dx, dy = target.x - basketball.x, target.y - basketball.y
    path_vector = pygame.math.Vector2(dx, dy)

    
    for player in players:
        if player not in exclusion_list:
            # Calculate vector from basketball to player
            to_player_vector = pygame.math.Vector2(player.x - basketball.x, player.y - basketball.y)
            
            # Check if the player is close to the direct path
            distance_to_path = path_vector.cross(to_player_vector) / path_vector.length()
            
            if abs(distance_to_path) < player.radius + basketball.radius:
                # Adjust the path to avoid the player
                avoidance_vector = path_vector.normalize().rotate(15)
                return avoidance_vector * MOVE_SPEED
    
    # If no adjustment needed, return the original path vector
    return path_vector.normalize() * MOVE_SPEED


In [425]:
# Initial circle placement

def is_valid_placement(new_circle, existing_circles):
    for circle in existing_circles:
        dx = new_circle.x - circle.x
        dy = new_circle.y - circle.y
        distance = math.sqrt(dx**2 + dy**2)
        
        # Calculate the minimum allowed distance between circle centers
        min_allowed_distance = circle.radius + new_circle.radius - 0.2 * min(circle.radius, new_circle.radius)
        if distance < min_allowed_distance:
            return False
    return True

#Stop attempt to create players after 1000 attempts 

def place_circle_with_constraints(existing_circles, radius, color):
    attempts = 0
    while attempts < 1000:  # Limit attempts to prevent infinite loop
        new_circle = Player(random.randint(radius, SCREEN_WIDTH - (2*radius)),
                            random.randint(radius, SCREEN_HEIGHT - (2*radius)),
                            radius, color)
        if is_valid_placement(new_circle, existing_circles):
            return new_circle
        attempts += 1
        
    raise Exception("Failed to place a new player without exceeding overlap threshold.")

In [426]:
# Colliding of circles

def circles_overlap(circle1, circle2, minimum_overlap_percentage):
    # Calculate the distance between the centers of the two circles
    distance_centers = math.sqrt((circle1.x - circle2.x) ** 2 + (circle1.y - circle2.y) ** 2)
    
    # Calculate the sum of the radii
    sum_of_radii = circle1.radius + circle2.radius
    
    # Overlap Percentage
    adjusted_distance_for_overlap = sum_of_radii * (1 - minimum_overlap_percentage)

    return distance_centers <= adjusted_distance_for_overlap

In [427]:
# Import Basketball Court image

#Displayed Screen Dimensions
SCREEN_WIDTH = 500
SCREEN_HEIGHT = 500

#Load and Scale image
image_location = '/Users/abhishekramesh/Library/Mobile Documents/com~apple~CloudDocs/NBA Analysis Project/NBA Court Diagram.jpg'

background_image = pygame.image.load(image_location)
background_image = pygame.transform.scale(background_image, (SCREEN_WIDTH, SCREEN_HEIGHT))

In [428]:
# Simulation Parameters

# Simulation Constants
NUM_PLAYERS = 10
PLAYER_RADIUS = 20
BALL_RADIUS = 10
COLOR_BLUE = (0, 0, 255)
COLOR_RED = (255, 0, 0)
COLOR_ORANGE = (255, 165, 0)
COLOR_WHITE = (255, 255, 255)
FPS = 60
MOVE_SPEED = np.random.normal(5.6, 1) #Speed at which basketball moves to the next player - Normal Distribution (mu = 5, sigma = 4/6)

# Simulation Variables
clock = pygame.time.Clock()
pass_timer = -1
pass_interval = random.uniform(0.5, 5)  #How many seconds before the player passes the ball
reached_player = True #When basketball is with a player
simulating = True #simulation is running
basketball_relative_x = 5 #X-axis displacemnt of the basketball compared to the player
basketball_relative_y = 5 #Y-axis displacemnt of the basketball compared to the player
basketball_player_overlap = 0.4 #When basketball and player overlap, set the relative distance between the two to prevent constant change
first_overlap = False #Calculate the basketball's relative x, y coordinate to the circle 
#global last_update_time
last_update_time = datetime.datetime.now() #set update time
dribble_timer = time.time()
dribble_switch_timer = time.time() - dribble_timer
oscillation_start_time = time.time()
basketball_displacement = 15 #Basketball displacement from the player

# Initialize Pygame
pygame.init()

# Create screen
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("NBA Game Simulation")

# Initialize players and basketball
players = []
for i in range(NUM_PLAYERS):
    color = COLOR_BLUE if i < NUM_PLAYERS // 2 else COLOR_RED
    player = place_circle_with_constraints(players, PLAYER_RADIUS, color)
    players.append(player)

# Choose a random blue player to place the basketball with
blue_players = [player for player in players if player.color == COLOR_BLUE]
current_player = random.choice(blue_players)

basketball = Basketball(
    current_player.x - basketball_relative_x, 
    current_player.y - basketball_relative_y, 
    BALL_RADIUS, COLOR_ORANGE
)

In [429]:
#Simulate the game

while simulating:
    
    #Create screen with basketball court as the background
    screen.blit(background_image, (0,0))

    #Stop simulation
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            simulating = False

    # Update and draw players
    for player in players:
        player.move(basketball)
        player.draw()
        
    current_time = time.time()
    time_elapsed = current_time - oscillation_start_time

    if not reached_player: # Move basketball towards the current player
        current_player.angle_update = False
        basketball.change_displacement = False
        
        # Create a new list excluding the current player and randomly choose a player to pass to
        blue_players_excluding_current = [player for player in blue_players if player != current_player]
        new_random_player = random.choice(blue_players_excluding_current)
        
        pass_players = [current_player, new_random_player] #2 players the ball is being passed between
        reached_player = move_basketball_to_player(basketball, current_player, players, pass_players, MOVE_SPEED)
        
        #Calculate the distance between the basketball and the receiving player
        if first_overlap == False:
            if circles_overlap(basketball, current_player, basketball_player_overlap):
                
                #Calculate x, y displacement relative to player
                basketball_relative_x = basketball.x - current_player.x
                basketball_relative_y = basketball.y - current_player.y
                
                reached_basketball_x = basketball_relative_x
                reached_basketball_y = basketball_relative_y
                last_update_time = datetime.datetime.now() #set update time
                first_overlap = True
                
    
    else: #Current player dribbles the basketball 
        basketball.update_position(current_player, time_elapsed)
        
        basketball.speed = current_player.speed
        basketball.angle = current_player.angle
        
        # Start timer once basketball reaches player
        if pass_timer < 0:
            pass_timer = 0  # Activate timer
        pass_timer += clock.get_time() / 1000.0  # Convert milliseconds to seconds
        
        # If pass_time greater or equal to pass_interval 
        if pass_timer >= pass_interval:
            # Time to pass the basketball to the next blue player
            current_index = blue_players.index(current_player) + 1
            if current_index >= len(blue_players):
                current_index = 0
            current_player = blue_players[current_index]
            pass_timer = -1  # Reset timer to deactivate
            first_overlap = False
            reached_player = False
    
    basketball.draw()  # Draw the basketball
    
    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()


KeyboardInterrupt: 

In [ ]:
#Upcoming Implementations

'''
1. When dribbling the basketball to another position with the same player make it a smooth transition

2. Dribble should switch sides from time to time

3. If an opposing player is within 10 pixels of the current_player, other player should change directions
'''


#Things to Implement
'''
- Base Basketball Actions
//Steal
- If opposing player comes in contact with basketball
--> Opposing player takes control of the ball
--> Passing team = opposing team


//Shooting
- Shooting basketball into basket
--> After basketball passed 3 times, player shoot basketball moves into basket circle

- Made shot
--> Basketball falls straight through

- Missed shot
--> Basketball moves randomly within a X radius (not behind backboard)

//Rebound
- When shot misses, player goes for the rebound
--> Players closeset to the ball moves to the ball
--> First player to reach the ball, gets the ball

- Save videos of passes
--> Create snips of videos when basketball moves between 2 players

- Match play by play to simulation
--> Get NBA play by play data
--> Match play
'''

# DEBUG



In [ ]:
%reset